In [ ]:
from numpy.lib.function_base import percentile
import scipy.io
import pandas as pd
import csv
import numpy as np
import pandas as pd
import os
from PIL import Image
import subprocess
import cv2
import shutil
import random
random.seed(123)
import sys
import copy

In [ ]:
subject_path = '/UT Multiview/subject/'
test_save_folder_path = ''
validation_save_folder_path = ''
train_save_folder_path = ''
synthe_all_img_save_folder_path = ''
save_subject_synthe_path = ''
save_subject_test_path = ''

test_csv = 'ut_multiview_test.csv'
validation_csv = 'ut_multiview_validation.csv'
train_csv = 'ut_multiview_train.csv'
all_csv =  'ut_multiview_synthe_all.csv'

os.makedirs(test_save_folder_path, exist_ok=True)
os.makedirs(validation_save_folder_path, exist_ok=True)
os.makedirs(train_save_folder_path, exist_ok=True)
os.makedirs(synthe_all_img_save_folder_path, exist_ok=True)
os.makedirs(save_subject_synthe_path, exist_ok=True)
os.makedirs(save_subject_test_path, exist_ok=True)
os.makedirs('personal_csv', exist_ok=True)

In [ ]:
subject_list = os.listdir(subject_path)
subject_list = sorted(subject_list)
print(subject_list)

for subject_folder_name in subject_list:
    subject_synthe_folder_path = os.path.join(save_subject_synthe_path, subject_folder_name)
    subject_test_folder_path = os.path.join(save_subject_test_path, subject_folder_name)

    os.makedirs(subject_synthe_folder_path, exist_ok=True)
    os.makedirs(subject_test_folder_path, exist_ok=True)

In [ ]:
def save_datas(path, subject_name):
    global img_cnt
    global gaze_data_cnt
    global subject_cnt

    if path[-4:] == "left":
        data_list = os.listdir(path)
        data_list = sorted(data_list)

        for data_name in data_list:
            file_name, ext = os.path.splitext(data_name)

            if ext == '.csv':
                csv_name = path + '/' + data_name
                # print(csv_name)

                with open(csv_name) as r_csv:
                    csv_datas = r_csv.readlines()
                    gaze_list = []

                    for csv_data in csv_datas:
                        line_data = csv_data
                        line_data_split = line_data.split(',')
                        gaze_x = line_data_split[0]
                        gaze_y = line_data_split[1]
                        gaze_z = line_data_split[2]
                        gaze = [gaze_data_cnt, gaze_x, gaze_y, gaze_z]
                        gaze_data_cnt += 1
                        gaze_list.append(gaze)

                    return gaze_list

            else:
                img_name = os.path.join(path, file_name) + '.bmp'
                shutil.copy(img_name, test_save_folder_path + str(img_cnt) + ".bmp")

                synthe_img_subject_path = os.path.join(save_subject_test_path, subject_name, str(subject_cnt)) +'.bmp'
                shutil.copy(img_name, synthe_img_subject_path)

                img_cnt += 1
                subject_cnt += 1        

def save_synthe_datas(path, subject_name):
    global train_img_cnt
    global validation_img_cnt
    global synthe_gaze_data_cnt
    global synthe_img_cnt
    global validation_img_list
    global subject_cnt

    distribute_img_cnt = 0
    gaze_data_cnt = 0

    # print(path)
    data_list = os.listdir(path)
    data_list = sorted(data_list)
    all_gaze_array = np.empty((1,4))
    validation_gaze_array = np.empty((1,4))
    train_gaze_array = np.empty((1,4))



    for data_name in data_list:
        file_name, ext = os.path.splitext(data_name)
        # print(data_name)

        if ext == '.csv':
            csv_name = path + '/' + data_name
            # print(csv_name)

            with open(csv_name) as r_csv:
                csv_datas = r_csv.readlines()
                # all_gaze_list = []
                # validation_gaze_list = []
                # train_gaze_list = []
                # all_gaze_array = np.empty((1,4))
                # validation_gaze_array = np.empty((1,4))
                # train_gaze_array = np.empty((1,4))
                # print(all_gaze_array.shape)

                validation_gaze_data_cnt = 0

                for csv_data in csv_datas:
                    # print('csv data: ', csv_data is not None)
                    line_data = csv_data
                    line_data_split = line_data.split(',')

                    gaze_x = line_data_split[0]
                    gaze_y = line_data_split[1]
                    gaze_z = line_data_split[2]

                    gaze = np.array([synthe_gaze_data_cnt, gaze_x, gaze_y, gaze_z])
                    synthe_gaze_data_cnt += 1

                    gaze = gaze[np.newaxis,:]
                    all_gaze_array = np.concatenate([all_gaze_array, gaze], axis=0)
                
                    if validation_gaze_data_cnt in validation_img_list:
                        validation_gaze_array = np.concatenate([validation_gaze_array, gaze], axis=0)
                        validation_gaze_data_cnt += 1
                    else:
                        train_gaze_array = np.concatenate([train_gaze_array, gaze], axis=0)
                        validation_gaze_data_cnt += 1
                # print(validation_gaze_data_cnt)
                all_gaze_array = np.delete(all_gaze_array,0,0)
                validation_gaze_array = np.delete(validation_gaze_array,0,0)
                train_gaze_array = np.delete(train_gaze_array,0,0)

                # print(all_gaze_array is not None, validation_gaze_array is not None, train_gaze_array is not None)
                # return all_gaze_array, validation_gaze_array, train_gaze_array


        else:
            img_name = os.path.join(path , file_name) + '.bmp'
            synthe_img_name = synthe_all_img_save_folder_path + str(synthe_img_cnt) + '.bmp' 
            shutil.copy(img_name, synthe_img_name)

            synthe_img_subject_path = os.path.join(save_subject_synthe_path, subject_name, str(subject_cnt)) +'.bmp'
            shutil.copy(img_name, synthe_img_subject_path)
            # print(synthe_img_subject_path)
            # sys.exit()

            synthe_img_cnt += 1
            subject_cnt += 1

            if distribute_img_cnt not in validation_img_list:
                train_img_name = train_save_folder_path + str(train_img_cnt) + '.bmp'
                shutil.copy(img_name, train_img_name)

                train_img_cnt += 1
                distribute_img_cnt += 1
            
            else:
                validation_img_name = validation_save_folder_path + str(validation_img_cnt) + '.bmp'
                shutil.copy(img_name, validation_img_name)

                validation_img_cnt += 1
                distribute_img_cnt += 1
    
    # print(all_gaze_array.shape)
    # sys.exit()
    return all_gaze_array, validation_gaze_array, train_gaze_array



    

In [ ]:
output_data_list = []
img_cnt = 1
gaze_data_cnt = 1

for subject_name in subject_list:
    folder_path = subject_path + subject_name + "/test_unzip/"
    file_list = os.listdir(folder_path)
    file_list.sort()
    subject_cnt = 1
    personal_output_data_list = []

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        # print(file_path)
        print(file_path)
        # sys.exit()
        gaze_data = save_datas(file_path, subject_name)

        if not gaze_data is None:
            output_data_list.append(gaze_data)
            personal_output_data_list.append(gaze_data)
    
    subject_test_csv = '/home/daiki/HDD_1/code/ut_multi-view_split_validation/personal_csv/ut_multiview_' + subject_name + '_test.csv'
    personal_output_data_array = np.array(personal_output_data_list).reshape(-1,4)
    df_personal_test = pd.DataFrame(personal_output_data_array)
    df_personal_test.to_csv(subject_test_csv,index=False,header = False)

print(np.array(output_data_list).shape)
print(np.array(output_data_list).reshape(-1,4).shape)
output_data_array = np.array(output_data_list).reshape(-1,4)
df = pd.DataFrame(output_data_array)
df.to_csv(test_csv,index=False,header = False)
print("Finish Output")

In [ ]:
print(subject_path)
print(subject_name)
print(file_path)

In [ ]:
synthe_all_data_list = []
synthe_validation_data_list = []
synthe_train_data_list = []
validation_gaze_data_list = []
train_gaze_data_list = []
synthe_img_cnt = 1
synthe_gaze_data_cnt = 1
validation_img_cnt = 1
train_img_cnt = 1
validation_img_list = [25, 50, 75, 100, 125]

# for i in range(5):
for subject_name in subject_list:
    # subject_name = subject_list[i]
    # print(subject_name)
    folder_path = subject_path + subject_name + "/synth_unzip/"
    synthe_file_list = os.listdir(folder_path)
    synthe_file_list.sort()
    subject_cnt = 1
    subject_synthe_all_data_list = []
    # print(len(synthe_file_list))

    for file_name in synthe_file_list:
        file_path = os.path.join(folder_path, file_name)
        if file_path[-4:] == "left":
            # print(file_path)
            all_gaze_data, validation_gaze_data, train_gaze_data = save_synthe_datas(file_path, subject_name)
            # print(all_gaze_data)

            synthe_all_data_list.append(all_gaze_data)
            subject_synthe_all_data_list.append(all_gaze_data)
            # print(len(synthe_all_data_list))
            synthe_validation_data_list.append(validation_gaze_data)
            synthe_train_data_list.append(train_gaze_data)
    
    subject_synthe_csv = '/home/daiki/HDD_1/code/ut_multi-view_split_validation/personal_csv/ut_multiview_' + subject_name + '_synthe.csv'

    subject_synthe_all_data_array = np.array(subject_synthe_all_data_list).reshape(-1,4)
    df_synthe_personal = pd.DataFrame(subject_synthe_all_data_array)
    df_synthe_personal.to_csv(subject_synthe_csv,index=False,header = False)

    # sys.exit()

print('all', np.array(synthe_all_data_list).shape)
print('all', np.array(synthe_all_data_list).reshape(-1,4).shape)

print('validation', np.array(synthe_validation_data_list).shape)
print('validation', np.array(synthe_validation_data_list).reshape(-1,4).shape)

print('train', np.array(synthe_train_data_list).shape)
print('train', np.array(synthe_train_data_list).reshape(-1,4).shape)

synthe_all_data_array = np.array(synthe_all_data_list).reshape(-1,4)
synthe_validation_data_array = np.array(synthe_validation_data_list).reshape(-1,4)
synthe_train_data_array = np.array(synthe_train_data_list).reshape(-1,4)

df_synthe_all = pd.DataFrame(synthe_all_data_array)
df_synthe_all.to_csv(all_csv,index=False,header = False)

df_synthe_validation = pd.DataFrame(synthe_validation_data_array)
df_synthe_validation.to_csv(validation_csv,index=False,header = False)

df_synthe_train = pd.DataFrame(synthe_train_data_array)
df_synthe_train.to_csv(train_csv,index=False,header = False)

print("Finish All Synthe Images Output")

In [ ]:
a ='abcdefghijk'
print(a[:5])